In [ ]:
import os
import json
import asyncio
from typing      import cast
from logging     import Logger, getLogger
from openai      import OpenAI

from azure.ai.inference.models import SystemMessage, UserMessage


from url_scraper import fetch_web

In [ ]:
# 環境変数の取得
AI_FOUNDRY_ENDPOINT = os.environ.get("AI_FOUNDRY_ENDPOINT")
AI_FOUNDRY_API_KEY  = os.environ.get("AI_FOUNDRY_API_KEY")
AI_FOUNDRY_MODEL    = os.environ.get("AI_FOUNDRY_MODEL")
MAX_TOKENS          = os.environ.get("MAX_TOKENS")
TEMPERATURE         = os.environ.get("TEMPERATURE")
TOP_P               = os.environ.get("TOP_P")

# メモ：
# ウィジット経由でのパラメータの取得方法では、無条件にパラメータの型がStringに変換されてしまう
# そのため、受け取ったパラメータを適切に型変換する必要がある
MAX_TOKENS  = int(MAX_TOKENS)
TEMPERATURE = float(TEMPERATURE)
TOP_P       = float(TOP_P)


# 簡易デバッグ用
print(f'AI_FOUNDRY_ENDPOINT: {AI_FOUNDRY_ENDPOINT}')
print(f'AI_FOUNDRY_API_KEY:  {AI_FOUNDRY_API_KEY}')
print(f'AI_FOUNDRY_MODEL:    {AI_FOUNDRY_MODEL}')
print(f'MAX_TOKENS:          {MAX_TOKENS}')
print(f'TEMPERATURE:         {TEMPERATURE}')
print(f'TOP_P:               {TOP_P}')

In [ ]:
logger    = getLogger(__name__)
semaphore = asyncio.Semaphore(10)

LP_URL   = "https://lp.br-lb.com/"

res_dict = await fetch_web(logger, semaphore, LP_URL)
print(res_dict)

In [ ]:
messages  = []
messages.append(SystemMessage(content=(
			"あなたは、商品記事の内容を分析し、マーケティングに必要な情報を抽出する専門家です。\n"
			"ユーザーが提供するスクレイピングされた商品情報を読み取り、以下の3つの観点で分析し、単語リスト（カンマ区切り、または箇条書き）として回答してください。\n\n"
			
			"1. 物理的な場所・環境：その商品が実際に使われる具体的な場所（例：リビング、キャンプ場、オフィスなど）\n"
			"2. 利用シーン・場面：どのような状況やタイミングで使われるか（例：朝のルーティン、テレワーク中、緊急時など）\n"
			"3. ユーザーの行動・心理特性：その商品を選ぶ人の特徴や目的（例：時短重視、健康志向、ミニマリストなど）\n\n"
			
			"【制約事項】\n"
			"・解説や文章は不要です。抽出した単語のみを列挙してください。\n"
			"・記事から直接読み取れない場合でも、商品の性質から論理的に推論できる内容は含めてください。"
        )))
messages.append(UserMessage(content=json.dumps(res_dict)))

llmClient = OpenAI(base_url=AI_FOUNDRY_ENDPOINT, api_key=AI_FOUNDRY_API_KEY)
response  = llmClient.chat.completions.create(
				messages=messages,
				tools=None,
				tool_choice=None,
				max_tokens=MAX_TOKENS,
				temperature=TEMPERATURE,
				top_p=TOP_P,
				model=AI_FOUNDRY_MODEL
			)

response.choices[0].message.content